# Defining nodes

In [48]:
import threading
import random
from time import sleep
from hashlib import sha256
import json
import time

In [55]:
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0

    def compute_hash(self):
        """
        A function that return the hash of the block contents.
        """
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()



In [89]:
class Blockchain:
    # difficulty of our PoW algorithm
    difficulty = 2

    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.peers = set()

    def create_genesis_block(self):
        """
        A function to generate genesis block and appends it to
        the chain. The block has index 0, previous_hash as 0, and
        a valid hash.
        """
        genesis_block = Block(0, [], 0, "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    @property
    def last_block(self):
        return self.chain[-1]

    def add_block(self, block, proof):
        """
        A function that adds the block to the chain after verification.
        Verification includes:
        * Checking if the proof is valid.
        * The previous_hash referred in the block and the hash of latest block
          in the chain match.
        """
        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            return False

        if not Blockchain.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        return True

    @staticmethod
    def proof_of_work(block):
        """
        Function that tries different values of nonce to get a hash
        that satisfies our difficulty criteria.
        """
        block.nonce = 0

        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()

        return computed_hash

    def add_new_transaction(self, transaction_type, node_type, count_of_product):
        transaction = {}
        transaction["type"] = transaction_type
        transaction["node_type"] = node_type
        transaction["count_of_product"] = count_of_product
        transaction["timestamp"] = time.time()

        self.unconfirmed_transactions.append(transaction)

    @classmethod
    def is_valid_proof(cls, block, block_hash):
        """
        Check if block_hash is valid hash of block and satisfies
        the difficulty criteria.
        """
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    @classmethod
    def check_chain_validity(cls, chain):
        result = True
        previous_hash = "0"

        for block in chain:
            block_hash = block.hash
            # remove the hash field to recompute the hash again
            # using `compute_hash` method.
            delattr(block, "hash")

            if not cls.is_valid_proof(block, block_hash) or \
                    previous_hash != block.previous_hash:
                result = False
                break

            block.hash, previous_hash = block_hash, block_hash

        return result
    
  
    def mine(self):
        """
        This function serves as an interface to add the pending
        transactions to the blockchain by adding them to the block
        and figuring out Proof Of Work.
        """
        if not self.unconfirmed_transactions:
            return False

        last_block = self.last_block

        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)

        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)

        self.unconfirmed_transactions = []

        return True


    def consensus(self):
        """
        Our naive consnsus algorithm. If a longer valid chain is
        found, our chain is replaced with it.
        """

        longest_chain = None
        current_len = len(self.chain)

        for node in self.peers:
            length = len(node.blockchain.chain)
            chain = node.blockchain
            if length > current_len and blockchain.check_chain_validity(chain):
                current_len = length
                longest_chain = chain

        if longest_chain:
            blockchain = longest_chain
            return True

        return False

    def announce_new_block(self, block):
        """
        A function to announce to the network once a block has been mined.
        Other blocks can simply verify the proof of work and add it to their
        respective chains.
        """
        block_data = json.dumps(block.__dict__, sort_keys=True)

        for peer in self.peers:
            block_new = Block(block_data["index"],
                        block_data["transactions"],
                        block_data["timestamp"],
                        block_data["previous_hash"],
                        block_data["nonce"])

            proof = block_data['hash']
            added = peer.add_block(block_new, proof)

            if not added:
                return "The block was discarded by the node"

            return "Block added to the chain"


    def mine_and_announce(self):
        result = self.mine()
        if not result:
            return "No transactions to mine"
        else:
            # Making sure we have the longest chain before announcing to the network
            chain_length = len(self.chain)
            #self.consensus()
            if chain_length == len(self.chain):
                # announce the recently mined block to the network
                self.announce_new_block(self.last_block)
            return "Block #{} is mined.".format(self.last_block.index)


    def get_chain(self):
        chain_data = []
        for block in self.chain:
            chain_data.append(block.__dict__)
        #print(chain_data)
        return json.dumps({"length": len(chain_data),
                        "chain": chain_data,
                        "peers": list(self.peers)})

    def create_chain_from_dump(chain_dump):
        generated_blockchain = Blockchain()
        generated_blockchain.create_genesis_block()
        for idx, block_data in enumerate(chain_dump):
            if idx == 0:
                continue  # skip genesis block
            block = Block(block_data["index"],
                        block_data["transactions"],
                        block_data["timestamp"],
                        block_data["previous_hash"],
                        block_data["nonce"])
            proof = block_data['hash']
            added = generated_blockchain.add_block(block, proof)
            if not added:
                raise Exception("The chain dump is tampered!!")
        return generated_blockchain


    def register_new_peers(self, blockchain_instance):
        
        # Add the node to the peer list
        self.peers.add(blockchain_instance)

        # Return the consensus blockchain to the newly registered node
        # so that he can sync
        return self.get_chain()

    def register_with_existing_node(self, blockchain_instance):
        """
        Internally calls the `register_node` endpoint to
        register current node with the node specified in the
        request, and sync the blockchain as well as peer data.
        """
        node_address = blockchain_instance
        
        response = self.register_new_peers(node_address)

        # update chain and the peers
        chain_dump = response.json()['chain']
        blockchain_instance = create_chain_from_dump(chain_dump)
        blockchain_instance.peers.update(response.json()['peers'])
        return "Registration successful"
     




In [90]:
bl = Blockchain()
bl2 = Blockchain()
#bl.peers.append(bl2)
bl.add_new_transaction("send", "warehouse", 4)
bl.create_genesis_block()
bl.mine()
bl.add_new_transaction("recieve", "warehouse", 5)
#bl.mine_and_announce()
bl.register_with_existing_node(bl2)
bl.get_chain()

TypeError: Object of type 'Blockchain' is not JSON serializable

In [5]:
class good:
    def __init__ (self, id):
        self.good_id = id

In [6]:
class Manufacturer:
    def __init__(self):
        self.num_goods = 0
    def produce(self):
        self.num_goods+=1
        good_produced = good(self.num_goods)
        print("Manufacturer produced good "+ str(good_produced.good_id))
        return good_produced

In [7]:
class Wholesaler:
    def __init__(self):
        self.curr_goods = 0
    def receive(self, good_received):
        self.curr_goods+=1
        print("Wholesaler received good " + str(good_received.good_id))
    def send(self, good_sent):
        self.curr_goods-=1
        print("Wholesaler sent good " + str(good_sent.good_id))

In [8]:
class Retailer:
    def __init__(self):
        self.stock_goods = 0
    def receive(self, good_received):
        self.stock_goods+=1
        print("Retailer received good " + str(good_received.good_id))

# Simulating the Supply Chain

In [9]:
man = Manufacturer()
who = Wholesaler()
ret = Retailer()

In [11]:
stock_of_goods = []  

In [12]:
def manufacture():
    for i in range(5):
        new_good = man.produce()
        stock_of_goods.append(new_good)
        t = random.randint(1,5)
        sleep(t/100)

In [13]:
def begin_chain ():
    cnt_completed = 0
    while (cnt_completed != 5):
        if (len(stock_of_goods) != 0):
            new_good = stock_of_goods.pop(0)
            who.receive(new_good)
            t = random.randint(1,5)
            sleep(t/100)
            who.send(new_good)
            t = random.randint(1,5)
            sleep(t/100)
            ret.receive(new_good)
            cnt_completed += 1

# Main function

In [14]:
t1 = threading.Thread(target=manufacture)
t2 = threading.Thread(target=begin_chain)
t1.start()
t2.start()
t1.join()
t2.join()
print("Ended")

Manufacturer produced good 1
Wholesaler received good 1
Wholesaler sent good 1
Manufacturer produced good 2
Retailer received good 1
Wholesaler received good 2
Manufacturer produced good 3
Wholesaler sent good 2
Retailer received good 2
Wholesaler received good 3
Wholesaler sent good 3
Manufacturer produced good 4
Manufacturer produced good 5
Retailer received good 3
Wholesaler received good 4
Wholesaler sent good 4
Retailer received good 4
Wholesaler received good 5
Wholesaler sent good 5
Retailer received good 5
Ended
